In [13]:
import pandas as pd
import  duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style("whitegrid")

%load_ext autoreload
from duckdb_flow.etl import preprocess_data, get_processed_files
from duckdb_flow.create_database import create_database
import duckdb

The autoreload module is not an IPython extension.


In [14]:
STAGING_PATH = "./data"
DATABASE_PATH = "./database/DDBB_duplicated.duckdb"

In [ ]:
# Connect to DuckDB
con = duckdb.connect(database=DATABASE_PATH, read_only=False)

con.execute('''
INSERT INTO movies (movie_id, primary_title, original_title, start_year, runtime_min, num_votes, label, subset, title_changed, title_length) 
VALUES 
    ('tt0012345', 'Movie A', 'Movie A', 2020, 120, 1000, TRUE, 'subset1', FALSE, 7),
    ('tt0012346', 'Movie B', 'Movie B', 2021, 110, 1500, FALSE, 'subset2', TRUE, 7),
    ('tt0012347', 'Movie C', 'Movie C', 2019, 105, 800, TRUE, 'subset3', FALSE, 7),
    ('tt0012348', 'Movie A', 'Movie A', 2020, 120, 1000, TRUE, 'subset1', FALSE, 7), -- Duplicated entry
    ('tt0012349', 'Movie B', 'Movie B', 2021, 110, 1500, FALSE, 'subset2', TRUE, 7), -- Duplicated entry
    ('tt0012350', 'Movie C', 'Movie C', 2019, 105, 800, TRUE, 'subset3', FALSE, 7); -- Duplicated entry
''')

con.close()

In [18]:
con = duckdb.connect(database=DATABASE_PATH, read_only=False)
movies = con.execute('''select * from movies''').fetch_df()
movies.shape
con.close()

In [6]:
movies

,movie_id,primary_title,original_title,start_year,runtime_min,num_votes,label,subset,title_changed,title_length,audit_time
0,tt0012345,Movie A,Movie A,2020,120,1000,True,subset1,False,7,2024-03-21 00:01:48.936
1,tt0012346,Movie B,Movie B,2021,110,1500,False,subset2,True,7,2024-03-21 00:01:48.936
2,tt0012347,Movie C,Movie C,2019,105,800,True,subset3,False,7,2024-03-21 00:01:48.936
3,tt0012348,Movie A,Movie A,2020,120,1000,True,subset1,False,7,2024-03-21 00:01:48.936
4,tt0012349,Movie B,Movie B,2021,110,1500,False,subset2,True,7,2024-03-21 00:01:48.936
5,tt0012350,Movie C,Movie C,2019,105,800,True,subset3,False,7,2024-03-21 00:01:48.936


In [11]:

# Connect to DuckDB
con = duckdb.connect(database=DATABASE_PATH, read_only=True)

# Run the query to detect duplicates with row counts over partitions
query = '''
SELECT * from (
SELECT
    movie_id,
    primary_title,
    original_title,
    start_year,
    runtime_min,
    num_votes,
    label,
    subset,
    title_changed,
    title_length,
    audit_time,
    ROW_NUMBER() OVER (PARTITION BY primary_title, original_title, start_year, runtime_min, num_votes, label, subset, title_changed, title_length ORDER BY audit_time) AS row_num
FROM
    movies) as t where row_num > 1
'''
duplicate_entries = con.execute(query).fetch_df()


In [12]:
duplicate_entries.head()

,movie_id,primary_title,original_title,start_year,runtime_min,num_votes,label,subset,title_changed,title_length,audit_time,row_num
0,tt0012350,Movie C,Movie C,2019,105,800,True,subset3,False,7,2024-03-21 00:01:48.936,2
1,tt0012348,Movie A,Movie A,2020,120,1000,True,subset1,False,7,2024-03-21 00:01:48.936,2
2,tt0012349,Movie B,Movie B,2021,110,1500,False,subset2,True,7,2024-03-21 00:01:48.936,2


In [17]:
con.close()